# Imports & Settings


In [1]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan
import wandb
import random
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost import XGBClassifier
# utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [2]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})
test_prep = pd.read_csv('data/test_prepared.csv', index_col='id', dtype={'type': str})
train_prep = pd.read_csv('data/train_prepared.csv', index_col='id', dtype={'type': str})

# tabnet


In [3]:
data = train_prep.copy()

In [4]:
features = data.drop('type', axis=1)
labels = data.type
# at least xgboost cannot deal with string labels
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(labels)
labels = label_encoder.transform(labels)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)

In [6]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

clf = TabNetClassifier()
clf.fit(
  X_train.values, y_train,
  eval_set=[(X_test.values, y_test)], patience=0
)

/home/till/.local/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/home/till/.local/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 4.08797 | val_0_accuracy: 0.16405 |  0:00:01s
epoch 1  | loss: 3.32842 | val_0_accuracy: 0.25619 |  0:00:03s
epoch 2  | loss: 2.89253 | val_0_accuracy: 0.26429 |  0:00:05s
epoch 3  | loss: 2.7657  | val_0_accuracy: 0.27    |  0:00:07s
epoch 4  | loss: 2.60369 | val_0_accuracy: 0.31405 |  0:00:09s
epoch 5  | loss: 2.3991  | val_0_accuracy: 0.31524 |  0:00:11s
epoch 6  | loss: 2.16148 | val_0_accuracy: 0.38571 |  0:00:13s
epoch 7  | loss: 2.02605 | val_0_accuracy: 0.46095 |  0:00:15s
epoch 8  | loss: 1.87415 | val_0_accuracy: 0.51357 |  0:00:17s
epoch 9  | loss: 1.74561 | val_0_accuracy: 0.55286 |  0:00:19s
epoch 10 | loss: 1.61665 | val_0_accuracy: 0.57167 |  0:00:21s
epoch 11 | loss: 1.4979  | val_0_accuracy: 0.57524 |  0:00:24s
epoch 12 | loss: 1.37125 | val_0_accuracy: 0.60857 |  0:00:26s
epoch 13 | loss: 1.2814  | val_0_accuracy: 0.59214 |  0:00:28s
epoch 14 | loss: 1.1886  | val_0_accuracy: 0.61024 |  0:00:30s
epoch 15 | loss: 1.1136  | val_0_accuracy: 0.63286 |  0

KeyError: 0

In [7]:
preds = clf.predict(X_test.values)
probs = clf.predict_proba(X_test.values)
val_acc = accuracy_score(y_test, preds)
val_acc

0.8516666666666667

In [10]:
clf.save_model('models/nn/first_try')

Successfully saved model at models/nn/first_try.zip


'models/nn/first_try.zip'